In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from scipy.stats import spearmanr

# 字体属性设定
font_prop = fm.FontProperties(fname='/mnt/data/file-ngwyeoEN29l1M3O1QpdxCwkj-sider-font.ttf')
plt.rcParams['axes.unicode_minus'] = False

# 读取数据（假设已整理成csv，格式见下方说明）
df = pd.read_excel('/mnt/data/data.csv')

# 列名示例：['Sample', 'Ipsi_Caudate', 'Ipsi_Putamen', ... 'CD4', 'CD8', ...]
# 设定脑区和细胞类型
regions = [
    ('Ipsilateral', 'Caudate'), ('Ipsilateral', 'Putamen'), ('Ipsilateral', 'Substantia nigra'),
    ('Contralateral', 'Caudate'), ('Contralateral', 'Putamen'), ('Contralateral', 'Substantia nigra'),
    ('Brainstem', ),
]
region_cols = [
    'Ipsi_Caudate', 'Ipsi_Putamen', 'Ipsi_Substantia_nigra',
    'Contra_Caudate', 'Contra_Putamen', 'Contra_Substantia_nigra',
    'Brainstem'
]
cell_types = ['CD4', 'CD8', 'IFN_CD8_CD8', 'Th1', 'Th2', 'TH17']

r_matrix = np.zeros((len(region_cols), len(cell_types)))
p_matrix = np.zeros_like(r_matrix)

# 计算Spearman相关和p
for i, reg in enumerate(region_cols):
    for j, cell in enumerate(cell_types):
        valid = df[[reg, cell]].dropna()
        r, p = spearmanr(valid[reg], valid[cell])
        r_matrix[i, j] = r
        p_matrix[i, j] = p

# 作图
fig, ax = plt.subplots(figsize=(10, 6))
im = ax.imshow(r_matrix, cmap='RdBu_r', vmin=-0.7, vmax=0.7)

# 添加文字
for i in range(len(region_cols)):
    for j in range(len(cell_types)):
        color = "black"
        # 相关系数
        ax.text(j, i-0.15, f"{r_matrix[i,j]:.2f}", ha='center', va='center', fontproperties=font_prop)
        # p值
        p_txt = f"{p_matrix[i,j]:.4f}"
        if p_matrix[i,j] < 0.05:
            p_txt = f"$\\bf{{{p_txt}}}$"
        ax.text(j, i+0.15, p_txt, ha='center', va='center', fontproperties=font_prop, color=color)

# 轴标签
ax.set_xticks(range(len(cell_types)))
ax.set_xticklabels(cell_types, fontproperties=font_prop, rotation=30, ha='right')
ax.set_yticks(range(len(region_cols)))
ax.set_yticklabels([' '.join(x) for x in regions], fontproperties=font_prop)

# 色条
cbar = plt.colorbar(im, ax=ax, fraction=0.025)
cbar.set_label("Spearman r", fontproperties=font_prop)
cbar.ax.tick_params(labelsize=10)

# 标题
ax.set_title("Spearman correlations (r colored; p below; bold if p<0.05)", fontproperties=font_prop)
fig.tight_layout()
plt.show()